In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dense, Conv2D, Wrapper
from tensorflow.keras.layers import ReLU, BatchNormalization, Flatten, MaxPool2D, Input
from tensorflow.python.keras.backend import symbolic_learning_phase
from tensorflow.python.keras.backend import eager_learning_phase_scope


#classes

class DropConnectDense(Dense):
    def __init__(self, *args, **kwargs):
        self.prob = kwargs.pop('prob', 0.5)
        if not 0. <= self.prob < 1.:
            raise NameError('prob must be at range [0, 1)]')


        super(DropConnectDense, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.in_feature = input_shape[-2]
        print(self.in_feature)
        
        super(DropConnectDense, self).build(input_shape)

    def call(self, inputs, train=True):
        if train:
            print('TRAIN')
            print(self.prob)
            print(tf.random.uniform((self.in_feature, self.units)))
            mask = tf.cast(tf.random.uniform((self.in_feature, self.units)) <= self.prob, tf.float32)
            kernel = tf.multiply(self.kernel, mask)
            output = tf.matmul(inputs, kernel)
        else:
            print('INFERENCE')
            output = tf.matmul(inputs, self.kernel * (1 - self.prob))
          
        if self.use_bias:
            output += self.bias
        return self.activation(output)
       

In [ ]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

print(type(x_train)) 

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

print(type(x_train))

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
X = tf.keras.layers.Input(shape=(784,))
x = DropConnectDense(units=128, prob=0.2, activation="relu", use_bias=True)(X)
x = DropConnectDense(units=64, prob=0.5, activation="relu", use_bias=True)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

None
TRAIN
0.2


TypeError: ignored

In [ ]:
model.summary()

In [ ]:
# Hyperparameters
batch_size=64
epochs=2


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

In [ ]:
# Run prediction the network
 with eager_learning_phase_scope(value=1):
    results = model.predict(x_test, batch_size=128, verbose = 2)
    print(results.shape)

## Testing section

In [3]:
import tensorflow as tf 
import tensorflow.keras.backend as K 

a = K.learning_phase()
print(a)



0
